In [15]:
import pandas as pd


In [16]:
# Read gene list file with cryptic exons inclusion levels
df_cryptic = pd.read_csv('PSI_results_CEs_total_cry_minus_C_40S_FP_wSJs_wide_WithSamples.csv', low_memory=False)

# Read the samples file
df_samples = pd.read_csv('samples_higher_10000000_column_star.uniquely_mapped_reads_number.tsv', sep='\t', low_memory=False)

# Read the TPM results CSV file into a DataFrame
df_tpm = pd.read_csv("tpm_results_log2.csv", low_memory=False) 


In [17]:
# remove noisy rows from df_cryptic

# Convert the '0' column to numeric, coercing errors to NaN
df_cryptic['0'] = pd.to_numeric(df_cryptic['0'], errors='coerce')

# Remove samples with non-numeric values in the '0' column
df_cryptic = df_cryptic.dropna(subset=['0'])

# Filter PSI's with 0's or 1's
df_cryptic = df_cryptic[(df_cryptic['0'] != 1) & (df_cryptic['0'] != 0)]

In [ ]:
# remove noisy rows from df_samples and identify tumor and normal samples

# Remove empty rows in 'cgc_sample_sample_type'
df_samples = df_samples.dropna(subset=['cgc_sample_sample_type'])

# Simplify the sample type to 'normal' or 'cancer'
df_samples['sample_type'] = df_samples['cgc_sample_sample_type'].apply(lambda x: 'normal' if x == 'Solid Tissue Normal' else 'cancer')

# Retrieve only columns of interest from df_samples
df_samples = df_samples[['rail_id', 'study', 'sample_type', 'gdc_cases.project.primary_site', 'gdc_cases.project.name']]

# Group by 'gdc_cases.project.primary_site' and filter groups that have both 'cancer' and 'normal' in 'sample_type'
df_samples = df_samples.groupby('gdc_cases.project.primary_site').filter(lambda x: set(x['sample_type']) == {'cancer', 'normal'})

In [19]:
# parse the TPM dataframe to only include rows of interest and format the table to prepare for relative expression analysis

# Filter TPM table to only include samples that are in the PSI table
df_tpm = df_tpm[df_tpm['sample_id'].isin(df_cryptic['level_1'])]

# Convert the 'TPM' column to numeric, coercing errors to NaN
df_tpm['TPM'] = pd.to_numeric(df_tpm['TPM'], errors='coerce')

# Remove samples with non-numeric values in the 'TPM' column
df_tpm = df_tpm.dropna(subset=['TPM'])

# Create a pivot table with sample_id as index and gene names as columns
df_tpm_pivot = df_tpm.pivot(index='sample_id', columns='gene_name', values='TPM')

# Reset the index to make 'sample_id' a column
df_tpm_pivot = df_tpm_pivot.reset_index()

# Rename the columns
df_tpm_pivot.columns.name = None
df_tpm_pivot.columns = ['sample_id'] + list(df_tpm_pivot.columns[1:])

In [20]:
# merge the dataframes to create a master dataframe
df_merged_tmp = pd.merge(df_tpm_pivot, df_samples, left_on='sample_id', right_on='rail_id', how='inner')

In [21]:
# For each group of rows in gdc_cases.project.primary_site, calculate the average expression for the hnRNPC gene for cases with sample_type 'cancer' and 'normal'.

# Group by primary site and sample type, calculate mean HNRNPC expression
grouped = df_merged_tmp.groupby(['gdc_cases.project.primary_site', 'sample_type'])['HNRNPC'].mean().unstack()
grouped = grouped.dropna()

# Calculate the ratio of cancer/normal for each primary site
grouped['cancer_vs_normal_ratio'] = grouped['cancer'] / grouped['normal']

# Define a new dataframe that contains the ratio of average hnRNPC expression in cancer vs normal for each primary site
# Reset index for a clean dataframe
hnRNPC_ratio_df = grouped.reset_index()

# sort table by lowest to highest ratio
hnRNPC_ratio_df = hnRNPC_ratio_df.sort_values(by='cancer_vs_normal_ratio')

# collect order of primary sites from hnRNPC_ratio_df
primary_site_order = hnRNPC_ratio_df['gdc_cases.project.primary_site'].tolist()


In [22]:
# merge cryptic exon and sample tables.
df_cryptic_samples = pd.merge(df_cryptic, df_samples, left_on='level_1', right_on='rail_id', how='left')


In [23]:
# Calculate medians for each group
medians = []
for site in primary_site_order:
    normal_median = df_cryptic_samples[
        (df_cryptic_samples['gdc_cases.project.primary_site'] == site) &
        (df_cryptic_samples['sample_type'] == 'normal')
    ]['0'].median()
    cancer_median = df_cryptic_samples[
        (df_cryptic_samples['gdc_cases.project.primary_site'] == site) &
        (df_cryptic_samples['sample_type'] == 'cancer')
    ]['0'].median()
    medians.append({'site': site, 'normal': normal_median, 'cancer': cancer_median})

median_df = pd.DataFrame(medians)

In [24]:
# Calculate the difference in median PSI (cancer - normal) for each primary site, ordered as in hnRNPC_ratio_df
median_diff = []
for site in hnRNPC_ratio_df['gdc_cases.project.primary_site']:
    row = median_df[median_df['site'] == site]
    if not row.empty:
        diff = row['cancer'].values[0] - row['normal'].values[0]
        median_diff.append(diff)
    else:
        median_diff.append(float('nan'))


# Obtain a summary dataframe that contains the ratios, median_diff, and gdc_cases.project.primary_site
summary_df = pd.DataFrame({
    'gdc_cases.project.primary_site': hnRNPC_ratio_df['gdc_cases.project.primary_site'],
    'cancer_vs_normal_ratio': hnRNPC_ratio_df['cancer_vs_normal_ratio'],
    'median_diff': median_diff
})
summary_df

,gdc_cases.project.primary_site,cancer_vs_normal_ratio,median_diff
14,Skin,0.951563,0.006734
0,Adrenal Gland,0.976510,0.012255
9,Kidney,0.982352,0.002963
19,Uterus,0.997609,0.007619
18,Thyroid,1.007273,0.003478
13,Prostate,1.007802,0.001812
5,Cervix,1.008373,-0.015060
2,Bladder,1.015545,0.006441
12,Pancreas,1.016133,-0.007619
6,Colorectal,1.020184,0.000000


In [25]:
import os
os.environ['R_HOME'] = 'C:/PROGRA~1/R/R-44~1.1' # this needs to match your R installation path
%load_ext rpy2.ipython


The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [26]:
%R -i summary_df

In [27]:
%%R

library(ggplot2)
library(cowplot)


# Calculate Spearman correlation
corr_test <- cor.test(summary_df$cancer_vs_normal_ratio, summary_df$median_diff, method = "spearman")
corr_val <- round(corr_test$estimate, 2)
p_val <- signif(corr_test$p.value, 2)

pdf("Corr_C-40S.pdf", width = 5, height = 5)

# Create the plot
p <- ggplot(summary_df, aes(x = cancer_vs_normal_ratio, y = median_diff*100, label = gdc_cases.project.primary_site)) +
    geom_smooth(method = "lm", color = "lightgray", se = FALSE, linetype = "solid") +
    geom_point(color = "#4a90e2", size = 2) + # C-iCLIP - "#a6c8ff" | C-40S - #4a90e2
    geom_text(vjust = -1, size = 2) +
    labs(
        x = "hnRNPC Cancer/Normal\nexpression Ratio",
        # y = "Median PSI difference (Cancer - Normal)\nfor C-iCLIP suppressed cryptic exons"
        y = "Median PSI difference (Cancer - Normal)\nfor C-40S suppressed cryptic exons"
    ) +
    geom_hline(yintercept = 0, linetype = "dotted", color = "black") +
    geom_vline(xintercept = 1, linetype = "dotted", color = "black") +
    annotate(
        "text",
        x = 1.01, # for C-40S
        y = -3.5, # for C-40S
        # x = 1.01, # for C-iCLIP
        # y = -1.5, # for C-iCLIP
        label = paste0("Spearman r = ", corr_val, "\np = ", p_val),
        hjust = 0, vjust = 0, size = 3.5, color = "black"#, fontface = "italic"
    ) +
    theme_cowplot()

print(p)

dev.off()



`geom_smooth()` using formula = 'y ~ x'
png 
  2 


In addition: Warning messages:
1: In cor.test.default(summary_df$cancer_vs_normal_ratio, summary_df$median_diff,  :
  Cannot compute exact p-value with ties
2: The following aesthetics were dropped during statistical transformation: label.
i This can happen when ggplot fails to infer the correct grouping structure in
  the data.
i Did you forget to specify a `group` aesthetic or to convert a numerical
  variable into a factor? 
